In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
def plot_candles(df_plot):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [3]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [18]:
df = df_raw[cols].copy()

In [22]:
def apply_top_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_h - row.mid_c
    else:
        return row.mid_h - row.mid_o

def apply_bottom_end_distance(row):
    if row.DIRECTION == 1:
        return row.mid_o - row.mid_l
    else:
        return row.mid_c - row.mid_l

def apply_stats(df):
    df['RANGE'] = df.mid_h - df.mid_l
    df['BODY_RANGE'] = abs(df.mid_c - df.mid_o)
    df['CENTER'] = (df.mid_h - df.mid_l) / 2 + df.mid_l
    df['BODY_PERC'] = df.BODY_RANGE / df.RANGE
    df['DIRECTION'] = df.mid_c - df.mid_o
    df['DIRECTION'] = df['DIRECTION'].apply(lambda x: 1 if x >= 0 else -1)
    df['DIST_TOP'] = df.apply(apply_top_end_distance, axis=1)
    df['DIST_BOTTOM'] = df.apply(apply_bottom_end_distance, axis=1)
    df['DIST_TOP_PERC'] = df.DIST_TOP / df.RANGE
    df['DIST_BOTTOM_PERC'] = df.DIST_BOTTOM / df.RANGE
    return df

In [23]:
df = apply_stats(df)

In [24]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,RANGE,BODY_RANGE,CENTER,BODY_PERC,DIRECTION,DIST_TOP,DIST_BOTTOM,DIST_TOP_PERC,DIST_BOTTOM_PERC
0,2018-01-01 22:00:00+00:00,112.666,112.793,112.582,112.723,0.211,0.057,112.6875,0.270142,1,0.070,0.084,0.331754,0.398104
1,2018-01-02 02:00:00+00:00,112.720,112.751,112.666,112.688,0.085,0.032,112.7085,0.376471,-1,0.031,0.022,0.364706,0.258824
2,2018-01-02 06:00:00+00:00,112.691,112.700,112.249,112.252,0.451,0.439,112.4745,0.973392,-1,0.009,0.003,0.019956,0.006652
3,2018-01-02 10:00:00+00:00,112.254,112.282,112.086,112.124,0.196,0.130,112.1840,0.663265,-1,0.028,0.038,0.142857,0.193878
4,2018-01-02 14:00:00+00:00,112.122,112.304,112.058,112.285,0.246,0.163,112.1810,0.662602,1,0.019,0.064,0.077236,0.260163
